In [4]:
import torch

import torch.nn as nn

from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

from train_functions import get_data, check, test_pretrained_model

In [5]:
H, W = 128, 128
BATCH_SIZE = 64
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


train_transf = transforms.Compose([transforms.RandomRotation(degrees=5),
    transforms.RandomCrop(480), 
    transforms.ColorJitter(brightness = [0.8,1.2]),
    transforms.Resize((H, W)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()]
)

test_transf = transforms.Compose([transforms.Resize((H, W)),
                             transforms.ToTensor()])

train_dataset, test_dataset = get_data(reduced=True, train_transformation=train_transf, test_transformation=test_transf)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


print("The device is:", DEVICE)
print("Number of train images:", len(train_dataset))
print("Number of test images:", len(test_dataset))
print("Image shape:", train_dataset[0][0].shape)
print("Mask shape:", train_dataset[0][1].shape)


def BerHu(depth, depth_pred):
	absdiff = torch.abs(depth - depth_pred)
	C = 0.2*torch.max(absdiff).item()
	return torch.mean(torch.where(absdiff < C, absdiff,(absdiff*absdiff+C*C)/(2*C)))

The device is: cuda
Number of train images: 1111
Number of test images: 338
Image shape: torch.Size([3, 128, 128])
Mask shape: torch.Size([1, 128, 128])


In [6]:
num_epochs = 1

In [7]:
from collections import defaultdict

predictions = defaultdict(list)

### V1 ResNetE_Decoder

In [8]:
from v1_ResNetE_Decoder import ResNetE_Decoder

model = ResNetE_Decoder()
scores = [nn.L1Loss(), nn.MSELoss(), BerHu]
path_to_weights = './checkpoints/ResNetE_Decoder/v1_trained_on_50.pth'

/anaconda/envs/azureml_py38_PT_and_TF/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/anaconda/envs/azureml_py38_PT_and_TF/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
predictions_1, test_scores_1 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.0939; MSELoss(): 0.0194; <function BerHu at 0x7fd45bc80160>: 0.1044


In [10]:
for i in range(273, 279):
    predictions["RED v1"].append(predictions_1[i])

### V2 ResNetE_Decoder

In [11]:
from v2_ResNetE_Decoder import ResNetE_Decoder

model = ResNetE_Decoder()
path_to_weights = './checkpoints/ResNetE_Decoder/v2_trained_on_27.pth'

In [12]:
predictions_2, test_scores_2 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.0976; MSELoss(): 0.0190; <function BerHu at 0x7fd45bc80160>: 0.1102


In [13]:
for i in range(273, 279):
    predictions['RED v2'].append(predictions_2[i])

### ResNetE_Decoder v3

In [14]:
from v3_ResNetE_Decoder import ResNetE_Decoder

model = ResNetE_Decoder()
path_to_weights = './checkpoints/ResNetE_Decoder/v3_trained_on_50.pth'

In [15]:
predictions_3, test_scores_3 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.1006; MSELoss(): 0.0207; <function BerHu at 0x7fd45bc80160>: 0.1130


In [16]:
for i in range(273, 279):
    predictions['RED v3'].append(predictions_3[i])

### ResNet_UNet 50_L1

In [17]:
from ResNet_UNet import ResNet_UNet

model = ResNet_UNet()
path_to_weights = './checkpoints/ResNet_UNet/model_50_L1.pth'

In [18]:
predictions_4, test_scores_4 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.0845; MSELoss(): 0.0171; <function BerHu at 0x7fd45bc80160>: 0.0951


In [19]:
for i in range(273, 279):
    predictions['RU L1'].append(predictions_4[i])

### ResNet_UNet 49_Berhu

In [20]:
from ResNet_UNet import ResNet_UNet

model = ResNet_UNet()
path_to_weights = './checkpoints/ResNet_UNet/model_49_Berhu.pth'

In [21]:
predictions_5, test_scores_5 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.0878; MSELoss(): 0.0175; <function BerHu at 0x7fd45bc80160>: 0.1022


In [22]:
for i in range(273, 279):
    predictions['RU BerHu'].append(predictions_5[i])

### ResNet_UNet Custom3

In [23]:
from ResNet_UNet import ResNet_UNet

model = ResNet_UNet()
path_to_weights = './checkpoints/ResNet_UNet/model_50_Custom3.pth'

In [24]:
predictions_6, test_scores_6 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.1231; MSELoss(): 0.0299; <function BerHu at 0x7fd45bc80160>: 0.1403


In [25]:
for i in range(273, 279):
    predictions['RU C'].append(predictions_6[i])

### DenseNet UNet

In [26]:
from DenseNet_UNet import DenseNet_UNet

model = DenseNet_UNet()
path_to_weights = './checkpoints/DenseNet_UNet/model_100.pth'

In [27]:
predictions_8, test_scores_8 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.0972; MSELoss(): 0.0234; <function BerHu at 0x7fd45bc80160>: 0.1058


In [28]:
for i in range(273, 279):
    predictions['Dense U'].append(predictions_8[i])

### No Skip

In [32]:
from DE_NoSkip import DE_gio_noskip

model = DE_gio_noskip()
path_to_weights = './checkpoints/DE_gio_noskip/epoch_50.pth'

In [33]:
predictions_7, test_scores_7 = test_pretrained_model(model, path_to_weights, test_loader, num_epochs, DEVICE, scores)

Test Loss: L1Loss(): 0.0891; MSELoss(): 0.0179; <function BerHu at 0x7fd45bc80160>: 0.1001


In [34]:
for i in range(273, 279):
    predictions['DE_NoSkip'].append(predictions_7[i])

In [35]:
import matplotlib.pyplot as plt

In [36]:

def print_images(images, depth_masks, predictions):
    fig, ax = plt.subplots(len(images),len(predictions) + 2, figsize=(12,8), dpi=900)
    cols = ['Image', 'Ground Truth'] + list(predictions.keys())
    for a, col in zip(ax[0], cols):
        a.set_title(col)

    for i in range(len(images)):
        image = images[i].permute(1, 2, 0).numpy()
        ax[i][0].imshow(image)
        ax[i][0].axis('off')

        ax[i][1].imshow(depth_masks[i].squeeze())
        ax[i][1].axis('off')

        for j, k in enumerate(predictions):
            ax[i][j + 2].imshow(predictions[k][i].squeeze())
            ax[i][j + 2].axis('off')

    
    plt.show()

In [37]:
images = []
depth_masks = []
for i in range(273, 279):
    images.append(test_dataset[i][0])
    depth_masks.append(test_dataset[i][1])

print_images(images, depth_masks, predictions)